## Tensorflow
- `Tensorflow`가 예전에는 쓰기 어려운 모델이었음 (코딩할 줄 아는 사람들만 사용)
- 그래서 `pytorch`가 많이 쓰이다 보니, `Tensorflow`에서도 쉽게 사용할 수 있는 `Keras` 만듦
- `Tensorflow 2.0`에서는 `keras`와 합쳐진 `tf.keras.Model`이나 `Sequential` 많이 사용
- `Tensorflow`에서 train step, test step을 사용하는 class 구조는 `pytorch lightening`과 비슷
  - `pytorch lightening` : `pytorch`를 더 쉽게 사용하기 위한 library
- Optimizer : Tensorflow addon
  - https://www.tensorflow.org/addons/overview?hl=ko
  - https://github.com/tensorflow/addons
    - 여러 tensorflow 개발자들이 다양한 optimizer 구현 코드 업로드

## Hydra
- Facebook에서 제공하는 범용적인 configuration management tool
- 'hydra config' 검색 : https://hydra.cc/docs/intro/
- Omegaconf library를 기반으로 만들어짐
  - Documentation : https://omegaconf.readthedocs.io/en/2.1_branch/
  - https://github.com/omry/omegaconf : 이 개발자가 facebook에 가서 만든 게 hydra
  - MLP 같은 작은 모델은 `__init__(self, input_dim: int, h1_dim: int, h2_dim: int, out_dim: int)` 이런 식으로 써도 되지만, 모델이 커질수록 `init` 안에 들어가야 할 *argument 많아지고 관리가 어려워짐
    - 그래서 configuration tool을 이용해서 관리하는 것이 권장
    - tensorflow에 hyperparameter도 같은 역할이고 이건 tensorflow와 연동이 되는 장점이 있지만 wandB 등 다른 tool과 연동이 잘 안 되는 단점
    - omegaconf가 structure 관리에도 유리

## Efficient Network
- https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')
drive_project_root = '/content/drive/MyDrive/#fastcampus'
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
# pip install tensorflow_addons

In [ ]:
# pip install wandb

In [ ]:
# pip install omegaconf

In [ ]:
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from omegaconf import OmegaConf, DictConfig    # DictConfig is for time checking

import hydra
from hydra.core.config_store import ConfigStore

import tensorflow as tf
import tensorflow_addons as tfa

import wandb

In [ ]:
from config_utils_tf import flatten_dict, register_config, get_optimizer_element, get_callbacks

GPU 확인

In [ ]:
tf.config.list_physical_devices()

In [ ]:
!nvidia-smi

https://www.tensorflow.org/</br>
- https://www.tensorflow.org/overview/?hl=ko</br>
- 튜토리얼 : https://www.tensorflow.org/tutorials?hl=ko
- API > Tensorflow : 각 함수에 대한 설명
  - 구글에 'Tensorflow API 한글' 검색하면 번역본도 볼 수 있음

초보자용 vs 전문가용
- 수업에서는 전문가용으로 할 예정
- 초보자용에서 사용하는 Sequential 버전(순차적으로 build 하는 방법)에는 한계가 있기 때문
- 실제 현업/연구에서는 Sequential 거의 안 씀

## define gpu
- https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy
- This strategy is typically used for training on one machine with multiple GPUs.
- 아래 코드 결과 보면 GPU 0번 잡아서 가져옴

In [ ]:
# mirrored_strategy = tf.distribute.MirroredStrategy()

## make model

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self, cfg: DictConfig):
        super().__init__()

        # tensorflow는 pytorch와 다르게 flatten을 하지 않아도 되지만
        # pytorch와 비슷한 구조로 코딩하기 위해 여기서는 썼음
        self.flatten = tf.keras.layers.Flatten()

        # tf nn module vs keras module
        # 1) nn module : tensorflow 1.0에서 사용, 기능이 조금 더 많음
        # 2) keras : tensorflow 2.0에서 사용
        self.linear1 = tf.keras.layers.Dense(input_dim=cfg.input_dim, units=cfg.h1_dim)
        # self.linear2 = tf.keras.layers.Dense(input_dim=h1_dim, units=h2_dim)
        # -> 이렇게 써도 되지만, pytorh보다 keras는 flexibility가 있어서 input_dim 생략해도 알아서 인지함
        self.linear2 = tf.keras.layers.Dense(units=cfg.h2_dim)
        self.linear3 = tf.keras.layers.Dense(units=cfg.out_dim)
        self.relu = tf.nn.relu
    
    # tensorflow에서는 'training=Fasle' 구문 꼭 넣기를 권장함
    # 나중에 regularization에서 drop out 할 때 이 부분을 조절할 수 있어야 함
    # - 학습일 때는 켜고, evaluation 때는 끄고
    def call(self, input, training=False):
        x = self.flatten(input)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        out = self.linear3(x)
        out = tf.nn.softmax(out)  # output을 확률값으로 바꿈
        return out
    
    # GradientTape() 구현
    # 따로 구현해도 되지만 class 안에 이렇게 넣어주면 나중에 더 코드가 깔끔해짐
    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"loss": loss})
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        logs.update({"test_loss": loss})
        return logs

### Dropout

In [ ]:
# tf.keras.Model 대신 MLP 써도 됨
# 그러면 위에서 썼던 MLP class를 가져오는 것이니
# MLP 모델과 달라진 __init__, call만 코드 써서 일부 수정해 주고
# 그대로 써도 되는 train_step, test_step는 생략 가능
class MLPWithDropout(tf.keras.Model):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.linear1 = tf.keras.layers.Dense(input_dim=cfg.input_dim, units=cfg.h1_dim)
        self.linear2 = tf.keras.layers.Dense(units=cfg.h2_dim)
        self.linear3 = tf.keras.layers.Dense(units=cfg.out_dim)
        self.dropout = tf.keras.layers.Dropout(cfg.dropout_prob)
        self.relu = tf.nn.relu
    
    def call(self, input, training=False):
        x = self.flatten(input)
        x = self.relu(self.linear1(x))
        x = self.dropout(x, training=training)
        x = self.relu(self.linear2(x))
        x = self.dropout(x, training=training)
        out = self.linear3(x)
        out = tf.nn.softmax(out)
        return out

    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs     

### CNN
- init, call 부분은 다시 짜고 train_step, test_step은 위 코드 그대로 사용
- tf.kears.layers.Conv2D
  - https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
  - filters : channel 수 의미 (RGB : 3)

In [ ]:
class ConvBatchNormMaxPool(tf.keras.layers.Layer):

    def __init__(
        self,
        conv2d_filters,
        conv2d_kernel_size,
        conv2d_strides,
        conv2d_padding,
        maxpool2d_pool_size,
        maxpool2d_strides,
        maxpool2d_padding,
        ):
        super().__init__()
        self.conv2d =tf.keras.layers.Conv2D(
            filters=conv2d_filters,
            kernel_size=conv2d_kernel_size,
            strides=conv2d_strides,
            padding=conv2d_padding,
        )
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.maxpool2d = tf.keras.layers.MaxPool2D(
            pool_size=maxpool2d_pool_size,
            strides=maxpool2d_strides,
            padding=maxpool2d_padding,
        )

    def call(self, input):
        # Conv2D -> Batch Normalization -> Activation -> Maxpooling (-> Dropout)
        x = self.conv2d(input)
        x = self.batchnorm(x)
        x = tf.keras.activations.relu(x)   # 이것도 위에 option을 줘서 relu 등 여러 가지를 시도해 볼 수 있음
        out = self.maxpool2d(x)
        return out

class CNN(tf.keras.Model):

    def __init__(self, cfg: DictConfig):
        super().__init__()
        # Conv2D -> Batch Normalization -> Activation -> Maxpooling (-> Dropout)
        self.layer1 = ConvBatchNormMaxPool(
            cfg.layer_1.conv2d_filters,
            cfg.layer_1.conv2d_kernel_size,
            cfg.layer_1.conv2d_strides,
            cfg.layer_1.conv2d_padding,
            cfg.layer_1.maxpool2d_pool_size,
            cfg.layer_1.maxpool2d_strides,
            cfg.layer_1.maxpool2d_padding,
        )
        self.layer2 = ConvBatchNormMaxPool(
            cfg.layer_2.conv2d_filters,
            cfg.layer_2.conv2d_kernel_size,
            cfg.layer_2.conv2d_strides,
            cfg.layer_2.conv2d_padding,
            cfg.layer_2.maxpool2d_pool_size,
            cfg.layer_2.maxpool2d_strides,
            cfg.layer_2.maxpool2d_padding,
        )

        # Batch Layer classification을 위해 두 가지 방법 사용 가능
        # 방법1. Global pooling -> 바로 dense + output layer (softmax) 적용
        # 방법2. Flatten -> dense로 연결
        # 지금은 방법2를 시도해 볼 것임
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(cfg.fc_1.units)
        self.fc2 = tf.keras.layers.Dense(cfg.fc_2.units)
        self.fc3 = tf.keras.layers.Dense(cfg.fc_3.units)
        self.dropout = tf.keras.layers.Dropout(cfg.dropout_prob)
    
    def call(self, input, training=False):
        input = tf.expand_dims(input, -1)  # [batch * 28 * 28 * 1] : 이렇게 filter 차원을 추가해 줘야 convolutional layer shape에 맞게 됨
        x = self.layer1(input)
        x = self.layer2(x)
        x = self.flatten(x)
        x = self.fc1(x)                  # fully connect
        x = self.dropout(x, training)    # training을 넣어야 evaluation에서 정상 작동
        x = self.fc2(x)
        out = self.fc3(x)
        out = tf.nn.softmax(out)         # softmax를 여기 말고 나중에 loss식 계산할 때 해도 괜찮음
        return out

    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            # images = (batch * 28 * 28)
            # -> convolutional layer에서는 filter가 필요
            # -> def call 부분에서 input의 차원을 늘려줘야 함 : expand_dim
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs     

### Efficient Net
- https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/EfficientNetB0

In [ ]:
class EfficientNetFinetune(tf.keras.Model):

    def __init__(self, cfg: DictConfig):
        super().__init__()
        # getattr(A, B) : B에 매칭되는 object를 A에서 가져옴
        self.efficientnet = getattr(
            tf.keras.applications.efficientnet,
            cfg.efficient_net_model_name
        )(**cfg.kwargs)

        # Freeze
        self.efficientnet.trainable = cfg.efficient_net_weight_trainable

        # Resizing 안 해도 되는데 혹시 모르니 함
        # input을 image net과 size 맞추기 위함
        self.resize=tf.keras.layers.Resizing(224, 224)
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")
        self.out_dense = tf.keras.layers.Dense(units=cfg.classes)        
    
    def call(self, input, training=False):
        input = tf.expand_dims(input, -1)  # [batch, 28, 28, 1]
        x = self.resize(input)             # [batch, 224, 224, 1]

        # imagenet은 RGB 채널이 있기 때문에 channel을 1이 아닌 3으로 바꿔줘야 함
        # 아래 방법은 권장하지 않는 약간 무식한 방법 (axis=-2 : 채널 2개 추가)
        x = tf.stack([x, x, x], axis=-2)   # [batch, 224, 224, 3] 

        x = self.efficientnet(x, training=training)

        # build top
        x = self.avgpool(x)
        out = self.out_dense(x)
        out = tf.nn.softmax(out)
        return out

    def train_step(self, data):
        # pass
        images, labels = data
        
        with tf.GradientTape() as tape:
            # images = (batch * 28 * 28)
            # -> convolutional layer에서는 filter가 필요
            # -> def call 부분에서 input의 차원을 늘려줘야 함 : expand_dim
            outputs = self(images, training=True)
            preds = tf.argmax(outputs, 1)

            # 위에서 out이 softmax 안 거친 경우, 여기 넣을 때 softmax 처리 해줘야 함
            loss = self.compiled_loss(
                labels, outputs
            )

        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        # pass
        images, labels = data
        outputs = self(images, training=False)
        preds = tf.argmax(outputs, 1)
        loss = self.compiled_loss(
            labels, outputs
        )

        # update the metrics
        self.compiled_metrics.update_state(labels, preds)

        # return a dict mapping metrics names to current values
        logs = {m.name: m.result() for m in self.metrics}
        return logs 

## Configuration 정의

### data configuration

In [ ]:
data_fashion_mnist_cfg: dict = {
    "n_class": 10,
    "train_val_split": [0.9, 0.1],
    "train_val_shuffle": True,
    "train_val_shuffle_buffer_size": 1024,
    "test_shuffle": False,
    "tset_shuffle_buffer_size": 1024,
}

### model configuration

In [ ]:
model_mnist_mlp_cfg: dict = {
    "name": "MLP",
    "data_normalize": True,
    "input_dim": 28*28*1,
    "h1_dim": 128,
    "h2_dim": 64,
    "out_dim": 10,
}

model_mnist_mlp_with_dropout_cfg: dict = {
    "name": "MLPWithDropout",
    "data_normalize": True,
    "input_dim": 128*128*1,
    "h1_dim": 128,
    "h2_dim": 64,
    "out_dim": 10,
    "dropout_prob": 0.3,
}

model_mnist_cnn_cfg: dict = {
    "name": "CNN",
    "data_normalize": True,
    'layer_1' : {
        "conv2d_filters": 32,    # output filter : 아무 값이나 넣어도 됨
        "conv2d_kernel_size": [3, 3],  # 2D니까 2*2로 넣어줘야 함
        "conv2d_strides": [1, 1],
        # padding
        # same : shape을 유지하면서 padding
        # valid : no padding
        "conv2d_padding": "same",
        "maxpool2d_pool_size": [2, 2],
        "maxpool2d_strides": [2, 2],
        "maxpool2d_padding": "valid",
    },
    'layer_2' : {
        "conv2d_filters": 64,
        "conv2d_kernel_size": [3, 3],
        "conv2d_strides": [1, 1],
        "conv2d_padding": "valid",
        "maxpool2d_pool_size": [2, 2],
        "maxpool2d_strides": [1, 1],   # 너무 많은 정보가 사라지지 않게 [1, 1]로 줄임
        "maxpool2d_padding": "valid",      
    },
    'fc_1': {"units":512},
    'fc_2': {"units":128},   # classification을 위해 점점 줄여서
    'fc_3': {"units":10},    # 최종 class 개수
    "dropout_prob": 0.25,
}

model_mnist_efficient_finetune_cfg: dict = {
    "name": "EfficientNetFinetune",
    "data_normalize": False,
    "efficient_net_model_name": "EfficientNetB0",
    "classes": 10,
    "efficient_net_weight_trainable": False,   # True : 학습할 수 있는 parameter가 늘어남

    # 어떤 걸 입력해야 할 지 모르니 keyword argument 형태로 넣기
    "kwargs": {
        "include_top": False,
        "weights": "imagenet",
    }
}


### optimizer configuration

In [ ]:
adam_warmup_lr_sch_opt_cfg = {
    "optimizer": {
        "name": "Adam",
        "other_kwargs": {},
    },
    "lr_scheduler": {
        "name": "LinearWarmupLRSchedule",
        "kwargs": {
            "lr_peak": 1e-3,
            "warmup_end_steps": 1500,
        }
    }
}

# RAdam은 scheduler 필요 없었음
radam_no_lr_sch_opt_cfg = {
    "optimizer": {
        "name": "RectifiedAdam",
        "learning_rate": 1e-3,
        "other_kwargs": {},
    },
    "lr_scheduler": None
}

# train_cfg
train_cfg: dict = {
    "train_batch_size": 128,
    "val_batch_size": 32,
    "tset_batch_size": 32,
    "max_epochs": 50,
    "distribute_strategy": "MirroredStrategy",   # colab(notebook)이 아니고 다른 server에서 하면 다른 strategy 필요
}

_merged_cfg_presets = {
    "cnn_fashion_mnist_radam": {
        "data": data_fashion_mnist_cfg,
        "model": model_mnist_cnn_cfg,
        "opt": radam_no_lr_sch_opt_cfg,
        "train": train_cfg,
    },
    "mlp_with_dropout_fahion_mnist_adam_with_warmup_lr_schedule": {
        "data": data_fashion_mnist_cfg,
        "model": model_mnist_mlp_with_dropout_cfg,
        "opt": adam_warmup_lr_sch_opt_cfg,
        "train": train_cfg,
    }
}

### hydra composition ###
# clear hydra instance -> Jupyter 환경에서 할 때는 일단 instance clear 하기
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialization
hydra.initialize(config_path=None)    # yaml을 쓰고 있고 외부에서 하면 config_path 지정해야 함

# using_config_key = "cnn_fashion_mnist_radam"
using_config_key = "mlp_with_dropout_fahion_mnist_adam_with_warmup_lr_schedule"
cfg = hydra.compose(using_config_key)

# define & override log _cfg
model_name = cfg.model.name
run_dirname = "dnn-tutorial-fashion-mnist-runs-tf"
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{using_config_key}-{model_name}"
log_dir = os.path.join(drive_project_root, "runs", run_dirname, run_name)

log_cfg = {
    "run_name": run_name,
    "callbacks": {
        "TensorBoard": {
            "log_dir": log_dir,
            "update_freq": 1,
        },
        "EarlyStopping": {
            "patience": 3,
            "verbose": True,
        }
    },
    "wandb": {
        "project": "fastcampus_fashion_mnist_tutorials_tf",
        "name": run_name,
        "tags": ["fastcampus_fashion_mnist_tutorials_tf"],
        "reinit": True,
        "sync_tensorboard": True,
    }
}

# unlock struct of config & set log config
OmegaConf.set_struct(cfg, False)
cfg.log = log_cfg

# relock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

# save yaml
# with open(os.path.join(log_dir, "config.yaml")) as f:
# with open("config.yaml", "w") as f:
#     OmegaConf.save(cfg, f)

# This would open the file we saved above
# and tell you the result of the model and its configs (weights, ...)
# You can check it whenever you want
# OmegaConf.load()

In [ ]:
def get_distribute_strategy(strategy_name: str, **kwargs):
    return getattr(tf.distribute, strategy_name)(**kwargs)

distribute_strategy = get_distribute_strategy(cfg.train.distribute_strategy)

In [ ]:
with distribute_strategy.scope():

    # dataset 정의 =====================================================================
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # normalization
    if cfg.model.data_normalize:
        x_train = x_train/255.0
        x_test = x_test/255.0

    # train/val splits
    assert sum(cfg.data.train_val_split) == 1.0
    train_size = int(len(x_train) * cfg.data.train_val_split[0])
    val_size = len(x_train) - train_size

    # train, test dataset 정의
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

    if cfg.data.train_val_shuffle:
        dataset = dataset.shuffle(buffer_size=cfg.data.train_val_shuffle_buffer_size,)
    
    if cfg.data.test_shuffle:
        test_dataset = test_dataset.shuffle(buffer_size=cfg.data.test_shuffle_buffer_size,)

    # train dataset을 train과 validation으로 나누기
    # tensorflow에서는 아래와 같이 take, skip 사용해서 데이터 많이 나눔
    train_dataset = dataset.take(train_size)   # train은 dataset에서 train_size만큼 take하고
    val_dataset = dataset.skip(train_size)     # val은 전체 dataset에서 train_size만큼 skip하고 남은 것

    # 검증
    print(f'train total : {len(dataset)} (train : {len(train_dataset)}, validation : {len(val_dataset)})')
    print(f'test : {len(test_dataset)}')

    # dataloader 정의 ==================================================================
    train_batch_size = cfg.train.train_batch_size
    val_batch_size = cfg.train.val_batch_size
    test_batch_size = cfg.train.tset_batch_size

    # drop_remainder=True : memory size가 안 맞으면 error 나는 것 방지 (pytorch는 이런 것 자동으로 처리함 = tensorflow와 차이점)
    train_dataloader = train_dataset.batch(train_batch_size, drop_remainder=True)
    val_dataloader = val_dataset.batch(val_batch_size, drop_remainder=True)
    test_dataloader = test_dataset.batch(test_batch_size, drop_remainder=True)

sample_example = next(iter(train_dataloader))
print(sample_example)

## define model

LinearWarmupLRScheduler 하는 이유
- SGD는 다른 optimizer 대비 learning rate 값에 매우 민감
  - learning rate를 잘 setting 해야 성능이 좋게 나옴 (Adam보다 더 좋게 나오기도 함)
- 따라서 optimizer와 함께 learning rate도 tuning 하는 게 원래는 좋음
- 그러나 학습 속도가 너무 느려지는 단점

warmup을 하기 어려운 상황이면?
- Rectified Adam으로 먼저 테스트 해 보고, optimizer는 조절해도 거의 결과 비슷하게 나오니, 모델링 부분을 업데이트 해 보기
- Rectified Adam에도 tuning 할 수 있는 요소 많음
  - https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/RectifiedAdam

In [ ]:
# 모델 정의
def get_model(cfg: DictConfig):
    if cfg.model.name == 'CNN':
        model = CNN(cfg.model)
    elif cfg.model.name == "EfficientNetFinetune":
        model = EfficientNetFinetune(cfg.model)
    elif cfg.model.name == 'MLP':
        model = MLP(cfg.model)
    elif cfg.model.name == 'MLPWithDropout':
        model = MLPWithDropout(cfg.model)
    else:
        raise NotImplementedError
    
    return model

with distribute_strategy.scope():
    model = get_model(cfg)

    # define loss
    loss_function = tf.losses.SparseCategoricalCrossentropy()

    # define optimizer & scheduler
    optimizer, scheduler = get_optimizer_element(
        cfg.opt.optimizer, cfg.opt.lr_scheduler
    )

    model.compile(
        loss = loss_function,
        optimizer = optimizer,
        metrics = [tf.keras.metrics.Accuracy()],
    )

    # model build
    # 이 부분 생략해도 되지만 build를 해 놓으면 나중에 debugging하기 좋음 -> 권장
    # batch 1 : 임의로 설정
    # model.build((1, 28*28*1))
    model.build((1, 28, 28))      # This build code is for 'CNN'

# 만약 build 안 하고 summary 하면 build, fit을 하거나 input shape를 넣으라고 경고 뜸
# fit은 학습이기 때문에 무거운 감이 있고 빠르게 하기 위해 build 선호
model.summary()

## get callbacks

In [ ]:
callbacks = get_callbacks(cfg.log)

## wandb setup

- https://docs.wandb.ai/guides/integrations/tensorflow
- sync_tensorboard=True : tensorflow에 적혀있는 걸 wandb에 업로드

In [ ]:
# flatten_dict(cfg)   # 전부 flatten 하게 바꿔주는 함수 -> nested 구조를 모두 under bar 형태로 바꿈

In [ ]:
wandb.init(
    config= flatten_dict(cfg),
    **cfg.log.wandb
)

경로 잘 찾고 있는지 확인

In [ ]:
! ls /content/drive/MyDrive/\#fastcampus/runs/

In [ ]:
# tensorboard load하기 : load extension
%load_ext tensorboard

# 경로 지정 : terminal 문법이기 때문에 #을 # 그대로 인지하려면 앞에 '\' 써줘야 함
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/

model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=cfg.train.max_epochs,
    callbacks=callbacks,
)

## model testing

In [ ]:
model.evaluate(test_dataloader)

auc curve

In [ ]:
test_labels_list = []
test_preds_list = []
test_outputs_list = []

for i, (test_images, test_labels) in enumerate(tqdm(test_dataloader, position=0, leave=True, desc='testing')):
    with mirrored_strategy.scope():
        test_outputs = model(test_images)
    test_preds = tf.argmax(test_outputs, 1)

    final_outs = test_outputs.numpy()
    test_outputs_list.extend(final_outs)

    test_preds_list.extend(test_preds.numpy())
    test_labels_list.extend(test_labels.numpy())

test_preds_list = np.array(test_preds_list)
test_labels_list = np.array(test_labels_list)

test_accuracy = np.mean(test_preds_list == test_labels_list)
print(f'\nacc: {test_accuracy*100}%')

roc curve

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

fpr = {}
tpr = {}
thresh = {}
n_class = 10

for i in range(n_class):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels_list, np.array(test_outputs_list)[:, i], pos_label=i)

In [ ]:
fpr

plot

In [ ]:
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], linestyle="--", label=f"Class {i} vs Rest") 

plt.title("Multi-class ROC Curve")
plt.xlabel("Flase Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="best")
plt.show()

auc score
- multi class이기 때문에 multi_class, average option 안 넣어주면 error 발생

In [ ]:
auc_score = roc_auc_score(test_labels_list, test_outputs_list, multi_class="ovo", average="macro")

In [ ]:
print(f'auc score : {auc_score*100}')